In [1]:
import numpy as np
from syft.core.tensor.autodp.phi_tensor import PhiTensor
from syft.core.tensor.autodp.gamma_tensor import GammaTensor
from syft.core.tensor.fixed_precision_tensor import FixedPrecisionTensor as FPT
from syft.core.adp.data_subject_list import DataSubjectList
from syft.core.tensor.lazy_repeat_array import lazyrepeatarray as lra

/home/e/anaconda3/envs/Hagrid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Torch Dataset equivalent

We have to implement:
- padding -> DONE!
- random horizontal flip -> DONE!
- random vertical flip -> DONE!
- random rotation -> Done but not confident about resultant LRA, or what to do with 0 values. -> Might try training the model without random rotations to see how testing accuracy suffers.
- normalize -> DONE!

In [2]:
a = PhiTensor(child=np.random.random((5,5)),data_subjects=np.ones((5,5)), min_vals=0, max_vals=1)

In [3]:
a

PhiTensor(child=FixedPrecisionTensor(child=[[32320 13466 31539 53025 47558]
 [48092 46383 10064 64947  6397]
 [23758 54672 39889 31523 35489]
 [64122 55424 64644 44428  5558]
 [58499 23557 14045 58880 64741]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [4]:
a.pad(width=1)

PhiTensor(child=FixedPrecisionTensor(child=[[46383 48092 46383 10064 64947  6397 64947]
 [13466 32320 13466 31539 53025 47558 53025]
 [46383 48092 46383 10064 64947  6397 64947]
 [54672 23758 54672 39889 31523 35489 31523]
 [55424 64122 55424 64644 44428  5558 44428]
 [23557 58499 23557 14045 58880 64741 58880]
 [55424 64122 55424 64644 44428  5558 44428]]), min_vals=<lazyrepeatarray data: 0 -> shape: (6, 6)>, max_vals=<lazyrepeatarray data: 1 -> shape: (6, 6)>)

In [5]:
a.random_horizontal_flip()

PhiTensor(child=FixedPrecisionTensor(child=[[32320 13466 31539 53025 47558]
 [48092 46383 10064 64947  6397]
 [23758 54672 39889 31523 35489]
 [64122 55424 64644 44428  5558]
 [58499 23557 14045 58880 64741]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [6]:
a.random_vertical_flip()

PhiTensor(child=FixedPrecisionTensor(child=[[58499 23557 14045 58880 64741]
 [64122 55424 64644 44428  5558]
 [23758 54672 39889 31523 35489]
 [48092 46383 10064 64947  6397]
 [32320 13466 31539 53025 47558]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [7]:
a.random_rotation(degrees=20)

PhiTensor(child=FixedPrecisionTensor(child=[[32320 13466 31539 53025 47558]
 [48092 46383 10064 64947  6396]
 [23758 54672 39889 31523 35489]
 [64122 55424 64644 44428  5558]
 [58499 23557 14045 58880 64741]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [8]:
a.normalize(mean=0.5, std=0.5)

PhiTensor(child=FixedPrecisionTensor(child=[[  -896 -38604  -2458  40514  29580]
 [ 30648  27230 -45408  64358 -52742]
 [-18020  43808  14242  -2490   5442]
 [ 62708  45312  63752  23320 -54420]
 [ 51462 -18422 -37446  52224  63946]]), min_vals=<lazyrepeatarray data: -1.0 -> shape: ()>, max_vals=<lazyrepeatarray data: 1.0 -> shape: ()>)

In [9]:
a.child.decode()

array([[0.49316406, 0.20547485, 0.48124695, 0.80909729, 0.72567749],
       [0.73382568, 0.70774841, 0.15356445, 0.99101257, 0.09761047],
       [0.36251831, 0.83422852, 0.60865784, 0.48100281, 0.54151917],
       [0.97842407, 0.84570312, 0.98638916, 0.67791748, 0.08480835],
       [0.8926239 , 0.35945129, 0.21430969, 0.8984375 , 0.98786926]])

<hr>
<hr>

## Model Training


To Do:
- Implement the layers needed for a ConvNet for DP Tensors:
    - Conv layer
    - BatchNorm2D
    - LeakyReLU
    - AvgPool2d
    - Linear
- Do 1 forward pass with these layers
- Do 1 backprop with these layers

In [2]:
from syft.core.tensor.nn.conv_layers import torch_Conv2d

In [3]:
import cv2

img = cv2.imread("/home/e/Downloads/Dataset/10253/0/10253_idx5_x1001_y1001_class0.png")
print(img.shape)
img = cv2.resize(img, (50, 50))
print(img.shape, type(img))

(50, 50, 3)
(50, 50, 3) <class 'numpy.ndarray'>


In [4]:
dp_tensor = PhiTensor(child=img, data_subjects=[0]  , min_vals=0, max_vals=255)

In [5]:
torch_Conv2d(dp_tensor, in_channels=3, out_channels=32, kernel_size=3, padding=2)

Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))


tensor([[[[ 1.5162e+01,  4.3755e+01,  2.2240e+01,  ...,  2.9702e+01,
           -1.4624e-01, -1.7711e+01],
          [ 6.3013e+01,  7.0522e+01,  4.5632e+01,  ...,  3.5935e+01,
           -1.2504e+01, -4.0195e+01],
          [ 8.0442e+01,  5.2915e+01,  2.4697e+01,  ...,  2.9630e+01,
           -3.0281e+01, -3.9793e+01],
          ...,
          [ 5.4399e+01,  4.0355e+01,  2.2242e+01,  ...,  3.2172e+01,
           -2.8172e+01, -2.3831e+01],
          [ 5.9271e+01,  2.3017e+01,  1.6920e+01,  ..., -2.5918e+01,
           -5.9865e+01, -3.1803e+01],
          [-1.9895e-01, -2.8566e+01, -2.7807e+01,  ..., -1.5105e+01,
           -1.5251e+01,  2.1750e+01]],

         [[-5.4101e+00,  1.0549e+01,  6.9967e+00,  ..., -4.8198e+00,
            4.0194e+00, -1.5810e+01],
          [ 1.2845e+01,  2.7181e+01,  9.2242e+01,  ...,  6.9547e+01,
            7.7108e+01,  2.9921e+01],
          [-6.6876e+01, -7.1329e+01,  2.1408e+01,  ...,  3.1253e+01,
            7.7112e+01,  7.6493e+01],
          ...,
     

In [6]:
import torch

In [9]:
l = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)

In [10]:
l(torch.Tensor(img.reshape(1, *img.shape[::-1])))

tensor([[[[-2.3042e+01,  2.7390e+01,  6.6446e+01,  ...,  6.6376e+01,
            7.9981e+01,  4.7197e+01],
          [-1.6410e+01,  7.1250e+01,  9.0292e+01,  ...,  5.5535e+01,
            9.4056e+01, -2.1167e+00],
          [-2.4273e+01, -1.6777e+01, -4.7674e+01,  ..., -1.5800e+01,
           -1.3821e+01, -3.7504e+01],
          ...,
          [-2.7891e+01, -2.1303e+01, -3.7191e+01,  ..., -4.9144e+01,
           -5.6089e+00, -3.2263e+01],
          [ 7.7883e+00, -3.5994e+01, -1.4201e+02,  ..., -7.9820e+01,
           -8.9908e+01, -7.0355e+01],
          [-1.4897e+01, -7.2479e+01, -7.3918e+01,  ..., -1.3083e+02,
           -8.7157e+01, -3.0793e+01]],

         [[-3.7417e+01, -2.1506e+01, -5.5670e+01,  ..., -5.2487e+01,
           -1.4567e+01, -2.2919e+01],
          [-5.2863e+01, -6.9858e+01, -8.6171e+01,  ..., -1.0241e+02,
           -4.6934e+01, -3.0191e+01],
          [-2.2203e+01, -1.2119e+02, -1.4763e+02,  ..., -1.3827e+02,
           -1.1838e+02, -4.1265e+01],
          ...,
     

In [13]:
for i in l.parameters():
    print(i)

Parameter containing:
tensor([[[[ 7.1647e-02, -2.3705e-02, -1.2546e-01],
          [-1.4189e-01, -1.8682e-02,  7.0026e-02],
          [ 7.6735e-02,  8.3194e-02, -3.1979e-02]],

         [[-8.6299e-02, -1.8227e-01,  1.1647e-01],
          [-1.7930e-01,  9.9490e-02,  1.6393e-01],
          [ 1.8256e-01,  1.2832e-02, -1.0314e-01]],

         [[-1.5376e-01, -1.3687e-01, -8.4154e-02],
          [ 2.5389e-02,  1.0936e-01, -1.4804e-01],
          [-2.1134e-03,  1.7570e-01, -7.6908e-04]]],


        [[[-1.7034e-02, -1.8419e-01,  4.5129e-02],
          [-1.2241e-01, -1.2847e-01, -5.4074e-02],
          [-1.1269e-01, -3.1313e-02, -2.9357e-02]],

         [[ 6.3640e-02, -7.9127e-02,  5.0851e-02],
          [ 1.5699e-01,  4.4807e-02,  1.5113e-01],
          [ 1.8370e-02, -2.2532e-02, -1.1394e-01]],

         [[-3.2158e-02, -1.0758e-01,  2.2125e-02],
          [-5.4170e-02, -7.1938e-02, -1.5727e-01],
          [-7.1067e-02,  1.5041e-01, -6.4342e-02]]],


        [[[ 1.5473e-02,  1.5128e-01,  2.8789

In [39]:
a = torch.Tensor(np.ones((3, 50,50)))
a.shape

torch.Size([3, 50, 50])

In [41]:
a[0]

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [2]:
from typing import Union
def Conv2d(image: Union[PhiTensor, GammaTensor], out_channels, kernel, padding=0, strides=1):
    """
    TODO:
    - Modify to work with DP Tensors
    - Modify to work with multiple images in a tensor
    """

    if not isinstance(padding, int):
        raise TypeError(f"Padding must be an integer, not type: {type(padding)}")
    kernel = np.flipud(np.fliplr(kernel))

    x_kernel, y_kernel = kernel.shape
    x_img, y_img = image.shape  # TODO: Needs to be modified if >1 image per tensor

    x_out = int(((x_img - x_kernel + 2 * padding)/strides) + 1)
    y_out = int(((y_img - y_kernel + 2 * padding)/strides) + 1)
    output = np.zeros(x_out, y_out)

    if padding != 0:
        padded_img = np.zeros((x_img + padding * 2, y_img + padding * 2))
        padded_img[padding: -padding, padding: -padding] = image
    else:
        padded_img = image

    for y in range(image.shape[1]):
        if y > image.shape[1] - y_kernel:
            break
        if y % strides == 0:
            for x in range(image.shape[0]):
                if x > image.shape[0] - x_kernel:
                    break
                try:
                    if x % strides == 0:
                        output[x, y] = (kernel * padded_img[x: x + x_kernel, y: y + y_kernel]).sum()
                except:
                    break

    return output

In [10]:
import cv2
test_img = cv2.imread("test_img.jpeg")

In [ ]:
kern = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])


In [11]:
from torch.nn import Conv2d

In [14]:
b = Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)

In [24]:
b.kernel_size

(3, 3)

In [2]:
a = np.ones((3,4,5))

In [5]:
print(a.shape)
b = a.transpose((2,0,1))
b.shape

(3, 4, 5)


(5, 3, 4)

In [6]:
def preprocess(image: np.ndarray, padding: int) -> np.ndarray:
    output_img = image.copy()
    image2 = np.pad(image, (padding,) )
    data2 = image2.transpose((2, 0, 1))
    data2 = np.expand_dims(data2, axis=0)
    data2 /= 255
    return data2 

In [7]:
def torch_Conv2d(image: np.ndarray, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True):
    """ This function mimics the PyTorch API more closely. """
    w_in, h_in = image.shape[-2:]
    w_out, h_out = (w_in + 2 * padding - k) // stride + 1, (h_in + 2 * padding - k) // stride + 1
    
    preprocessed_img = preprocess(image, padding=padding)
    
    w = np.zeros((out_channels, in_channels, k, k))
    b = np.zeros((out_channels))

In [5]:
conv_result = Conv2d(test_img, out_channels=)

TypeError: Conv2d() missing 2 required positional arguments: 'out_channels' and 'kernel'

In [ ]:
def conv2D(image, out_channels, kernel_size, padding=0, strides=1):
    pass

In [9]:
from torchvision import transforms
import torch
import cv2

In [8]:
img = cv2.imread("/home/e/Downloads/Dataset/10253/0/10253_idx5_x1001_y1001_class0.png")
img = cv2.resize(img, (50, 50))
img.shape

(50, 50, 3)

In [16]:
pre_norm = transforms.ToTensor()(transforms.Pad(64, padding_mode="reflect")(transforms.ToPILImage()(img)))

In [20]:
pre_norm.mean()

tensor(0.7206)

In [21]:
pre_norm.std()

tensor(0.1548)

In [17]:
res = transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])(pre_norm)

In [18]:
res.mean()

tensor(0.4411)

In [19]:
res.std()

tensor(0.3096)

In [22]:
test = (pre_norm - pre_norm.mean()) / (pre_norm.std())

In [23]:
test.mean()

tensor(2.0548e-08)

In [24]:
test.std()

tensor(1.)

In [33]:
d = np.random.random((3, 5, 5,))

In [34]:
for i, v in enumerate(d):
    print(i, v)

0 [[0.02642461 0.56923667 0.22130778 0.31342289 0.11633154]
 [0.12364862 0.39458504 0.65386739 0.29517113 0.84490691]
 [0.70352203 0.29752624 0.11884652 0.83865271 0.64257607]
 [0.46191271 0.37322436 0.66233822 0.20258328 0.65426208]
 [0.0085115  0.73365111 0.01689818 0.3798477  0.56232386]]
1 [[0.44206425 0.67285731 0.61315491 0.37023281 0.39998368]
 [0.26211304 0.39814871 0.69543346 0.46826847 0.8743573 ]
 [0.41289791 0.7571391  0.96612177 0.79737452 0.2898907 ]
 [0.43762697 0.91868411 0.04907501 0.20053439 0.47844655]
 [0.98855054 0.17506782 0.69149578 0.49048932 0.98230032]]
2 [[0.23462324 0.10813466 0.40229739 0.49501227 0.54822249]
 [0.86502586 0.37189566 0.74206779 0.59223384 0.80024581]
 [0.23309564 0.32543315 0.75521786 0.42293359 0.69048595]
 [0.26698754 0.04232397 0.43985301 0.10540017 0.12507499]
 [0.20655178 0.51363049 0.60833977 0.15641146 0.16842811]]


In [35]:
d - np.ones((3, 1))

ValueError: operands could not be broadcast together with shapes (3,5,5) (3,1) 